## Join the Duet Server the Data Owner 1 connected to

In [1]:
import syft as sy

In [2]:
duet1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [51]:
duet2 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


## Get data from Duet Server

In [81]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 27034ee5bb12433b9a5ad4f8d292f7a2>,[X_Train_Owner1],"Dataset of 400 samples, 7 feature",<class 'torch.Tensor'>
1,<UID: 6a775e86c0534b03a3acebdead61c60f>,[y_Train_Owner1],"Dataset of 400 samples, 1 feature",<class 'torch.Tensor'>
2,<UID: 65072e4612e44795b753a24d66cd0d66>,[X_Test_Owner1],"Dataset of 400 samples, 7 feature",<class 'torch.Tensor'>
3,<UID: 3f08d213eece4aaa8b5a2d281095c2fc>,[y_Test_Owner1],"Dataset of 400 samples, 1 feature",<class 'torch.Tensor'>


In [22]:
X_Train_Owner1_ptr = duet1.store[0]
y_Train_Owner1_ptr = duet1.store[1]
X_Test_Owner1_ptr = duet1.store[2]
y_Test_Owner1_ptr = duet1.store[3]

print(X_Train_Owner1_ptr.tags)
print(X_Train_Owner1_ptr.description)

['X_Train_Owner1']
Dataset of 400 samples, 7 feature


In [82]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 7083551ae5334fc99a22f4b43d44c9b7>,[X_Train_Owner2],"Dataset of 400 samples, 7 feature",<class 'torch.Tensor'>
1,<UID: a45df775e0744a169e1fbbfa7fcbdc03>,[y_Train_Owner2],"Dataset of 400 samples, 1 feature",<class 'torch.Tensor'>
2,<UID: 030c2da760bd4e9e8cd32f19c380988d>,[X_Test_Owner2],"Dataset of 400 samples, 7 feature",<class 'torch.Tensor'>
3,<UID: 245b75ac7b1648f59352636b13d7044f>,[y_Test_Owner2],"Dataset of 400 samples, 1 feature",<class 'torch.Tensor'>


In [53]:
X_Train_Owner2_ptr = duet2.store[0]
y_Train_Owner2_ptr = duet2.store[1]
X_Test_Owner2_ptr = duet2.store[2]
y_Test_Owner2_ptr = duet2.store[3]

print(X_Train_Owner2_ptr.tags)
print(X_Train_Owner2_ptr.description)

['X_Train_Owner2']
Dataset of 400 samples, 7 feature


## Build a Base Classifier Model

In [36]:
class SyNet(sy.Module):
    def __init__(self,input_channels,output_channels,torch_ref):
        super(SyNet,self).__init__(torch_ref=torch_ref)
        #Our network:
        # Linear1->relu->Batchnorm->Linear2->relu->Batchnorm->Dropout->Linear3->output
        # Softmax is added in the predict function
        #This applies Linear transformation to input data. 
        self.fc1 = self.torch_ref.nn.Linear(input_channels,int(1.5*input_channels))
        self.fc2 = self.torch_ref.nn.Linear(int(1.5*input_channels),int(1.5*input_channels))
        self.fc3 = self.torch_ref.nn.Linear(int(1.5*input_channels),output_channels)
        
        self.relu = self.torch_ref.nn.ReLU()
        self.dropout = self.torch_ref.nn.Dropout(p=0.1)
        self.batchnorm1 = self.torch_ref.nn.BatchNorm1d(int(1.5*input_channels))
        self.batchnorm2 = self.torch_ref.nn.BatchNorm1d(int(1.5*input_channels))
        self.sigmoid = self.torch_ref.nn.Sigmoid()
    #This must be implemented
    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.relu(self.fc2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
    def predict(self,x):
        output = self.forward(x)
        prediction = self.torch_ref.argmax(output,1)
        return prediction

In [48]:
def classifier_train(epochs, model, optimizer, X_ptr,y_ptr,criterion,torch_ref):
    losses = []
    for i in range(epochs):
        #Precit the output for Given input
        y_pred_ptr = model.forward(X_ptr)
        #Compute Cross entropy loss
        loss = criterion(y_pred_ptr,y_ptr)
        loss_item = loss.item()
        #Request to get the loss value
        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )
        #Add loss to the list
        losses.append(loss_value)
        #Print loss
        if i%50==0:
            print("Epoch:",i," Loss:",loss_value)
        
        #Clear the previous gradients
        optimizer.zero_grad()
        #Compute gradients
        loss.backward()
        #Adjust weights
        optimizer.step()
    return losses

In [99]:
import torch as base_torch         
import numpy as np

In [39]:
# Base Classifier Model
X_Train_Owner1_shape = X_Train_Owner1_ptr.shape.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )
input_channels = X_Train_Owner1_shape[1]
output_channels = 2
base_model = SyNet(input_channels,output_channels,base_torch)

## Send copy of model to Data Owner1and train them remotely

In [50]:
remote_model1 = base_model.send(duet1)

remote_torch1 = duet1.torch
params = remote_model1.parameters()
#Define loss criterion
criterion = remote_torch1.nn.CrossEntropyLoss()
#Define the optimizer
optimizer = remote_torch1.optim.Adam(params=params, lr=0.001)
#Number of epochs
epochs = 1000

# Training
losses = classifier_train(epochs, remote_model1, optimizer, X_Train_Owner1_ptr, y_Train_Owner1_ptr,criterion, remote_torch1)

Epoch: 0  Loss: 0.6714057326316833
Epoch: 50  Loss: 0.6365100145339966
Epoch: 100  Loss: 0.6091839075088501
Epoch: 150  Loss: 0.5800541043281555
Epoch: 200  Loss: 0.5645753741264343
Epoch: 250  Loss: 0.5452878475189209
Epoch: 300  Loss: 0.5264979004859924
Epoch: 350  Loss: 0.5114352107048035
Epoch: 400  Loss: 0.49581339955329895
Epoch: 450  Loss: 0.4737929105758667
Epoch: 500  Loss: 0.46566444635391235
Epoch: 550  Loss: 0.4550821781158447
Epoch: 600  Loss: 0.4539676308631897
Epoch: 650  Loss: 0.44768208265304565
Epoch: 700  Loss: 0.4459100663661957
Epoch: 750  Loss: 0.43706202507019043
Epoch: 800  Loss: 0.4346407353878021
Epoch: 850  Loss: 0.4331651031970978
Epoch: 900  Loss: 0.42970994114875793
Epoch: 950  Loss: 0.4294106960296631


In [84]:
#y_Test_Owner1_predict_ptr = remote_model1.predict(X_Test_Owner1_ptr)
#s = y_Test_Owner1_predict_ptr-y_Test_Owner1_ptr
#res = s.get(reason="To evaluate training progress",
#             request_block=True,
#             timeout_secs=5,
#         )

## Send copy of model to Data Owner2and train them remotely

In [54]:
remote_model2 = base_model.send(duet2)

remote_torch2 = duet2.torch
params = remote_model2.parameters()
#Define loss criterion
criterion = remote_torch2.nn.CrossEntropyLoss()
#Define the optimizer
optimizer = remote_torch2.optim.Adam(params=params, lr=0.001)
#Number of epochs
epochs = 1000

# Training
losses = classifier_train(epochs, remote_model2, optimizer, X_Train_Owner2_ptr, y_Train_Owner2_ptr,criterion, remote_torch2)

Epoch: 0  Loss: 0.6677805185317993
Epoch: 50  Loss: 0.6347150206565857
Epoch: 100  Loss: 0.6062013506889343
Epoch: 150  Loss: 0.5813589692115784
Epoch: 200  Loss: 0.5615498423576355
Epoch: 250  Loss: 0.5425483584403992
Epoch: 300  Loss: 0.5360767245292664
Epoch: 350  Loss: 0.5218856334686279
Epoch: 400  Loss: 0.5140787363052368
Epoch: 450  Loss: 0.5067965984344482
Epoch: 500  Loss: 0.5043188333511353
Epoch: 550  Loss: 0.49677520990371704
Epoch: 600  Loss: 0.4892673194408417
Epoch: 650  Loss: 0.487337201833725
Epoch: 700  Loss: 0.4887542426586151
Epoch: 750  Loss: 0.48170891404151917
Epoch: 800  Loss: 0.4763626754283905
Epoch: 850  Loss: 0.4875952899456024
Epoch: 900  Loss: 0.47301235795021057
Epoch: 950  Loss: 0.47354790568351746


## Averaging Model Updates

In [88]:
from collections import OrderedDict

In [87]:
param1 = remote_model1.parameters().get(request_block=True)
param2 = remote_model2.parameters().get(request_block=True)

print("Base Model parameters:")
print(base_model.parameters())
print()

print("Remote model1 parameters:")
print(param1)
print()

print("Remote model2 parameters:")
print(param2)

Base Model parameters:
[Parameter containing:
tensor([[ 0.1080,  0.3046,  0.2469, -0.1963, -0.1279,  0.2022, -0.2835],
        [-0.3687, -0.0809,  0.2325,  0.2654, -0.0539,  0.3263,  0.0971],
        [ 0.1901, -0.0424,  0.1180, -0.2705,  0.0703,  0.2696, -0.3205],
        [ 0.1972, -0.1518,  0.1409,  0.3067, -0.1083,  0.1113, -0.0007],
        [ 0.1251,  0.1007, -0.2291,  0.0200,  0.3618,  0.1570,  0.1333],
        [-0.2548, -0.0458,  0.1391, -0.1971,  0.3332, -0.0134,  0.0690],
        [ 0.3161,  0.1424,  0.2739, -0.3420,  0.1806,  0.0810, -0.0678],
        [-0.3193, -0.2309, -0.2773,  0.3722, -0.1205, -0.1357, -0.0206],
        [-0.0185, -0.3740,  0.0416, -0.2869, -0.0853,  0.0132,  0.1743],
        [ 0.3443, -0.3391,  0.0027,  0.0313, -0.3370, -0.2535,  0.2128]],
       requires_grad=True), Parameter containing:
tensor([-2.1954e-01, -2.5332e-04, -2.3299e-02, -3.5002e-01,  3.7355e-01,
        -2.7745e-01, -1.8291e-01, -2.7888e-01, -1.0975e-01, -1.5595e-01],
       requires_grad=True)

In [89]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

OrderedDict([('fc1.weight', tensor([[ 0.0779,  0.0569,  0.2021, -0.1684, -0.0360,  0.2680, -0.1703],
        [-0.2216, -0.1781,  0.1602,  0.0738, -0.0785,  0.3156,  0.2587],
        [ 0.1028, -0.0767, -0.0728, -0.2424,  0.0311,  0.3255, -0.2238],
        [ 0.2700, -0.2687,  0.2896,  0.1487,  0.0590, -0.2297,  0.0506],
        [ 0.0889, -0.0486,  0.0543,  0.0124,  0.5594,  0.1218,  0.0915],
        [-0.2139,  0.1943, -0.0509,  0.1414,  0.2502, -0.2040,  0.1275],
        [ 0.1308,  0.0856,  0.3098, -0.1076,  0.2109,  0.0107, -0.1479],
        [ 0.1113, -0.3909, -0.0487,  0.1316,  0.0051, -0.0901,  0.3607],
        [-0.1767, -0.1866, -0.1866, -0.3488, -0.0737,  0.1476,  0.0562],
        [ 0.3401, -0.3482,  0.0943, -0.0443, -0.2453, -0.2647,  0.0708]])), ('fc1.bias', tensor([-0.4654, -0.3129,  0.3822, -0.0754,  0.4590, -0.3758, -0.4711, -0.0207,
        -0.2039, -0.2992])), ('fc2.weight', tensor([[ 0.0737, -0.0900, -0.0814,  0.0586,  0.0841, -0.0670, -0.3504, -0.0583,
         -0.0751, -0.

In [103]:
# Get Average Model
avg_updates = OrderedDict()
for key in remote_model1_updates.keys():
    avg_updates[key] = (remote_model1_updates[key] + remote_model2_updates[key]) / 2

    
combined_model = SyNet(input_channels,output_channels,base_torch)

combined_model.load_state_dict(avg_updates)

## Test the prediction of combined model

In [115]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

german = pd.read_csv("./data/german_data_2.csv")  

In [116]:
test_size = 0.20

processed_data = None
categorical = None
label_encoders = {}

def preprocessing(dataset, data, test_size):
    """
    Preprocess dataset

    Parameters
    ----------
    data: DataFrame
        Pandas dataframe containing German dataset.
    """
    
    global processed_data
    global categorical
    global label_encoders

    # Reset global variables
    
    processed_data = None
    categorical = None
    label_encoders = {}


    if dataset == "German":
        # Drop savings account and checkings account columns as they contain a lot
        # of NaN values and may not always be available in real life scenarios
        data = data.drop(columns = ['Saving accounts', 'Checking account'])
        
    dat_dict = data.to_dict()
    new_dat_dict = {}

    # rename columns(Make them lowercase and snakecase)
    for key, value in dat_dict.items():
        newKey = key
        if type(key) == str:
            newKey = newKey.lower().replace(' ', '_')
        # if newKey != key:
        new_dat_dict[newKey] = dat_dict[key]
    del dat_dict

    data = pd.DataFrame.from_dict(new_dat_dict)
    del new_dat_dict


    # print(data.describe())
    # print(data.describe(include='O'))

    cols = data.columns
    num_cols = data._get_numeric_data().columns
    categorical = list(set(cols) - set(num_cols))

    # Drop null rows
    data = data.dropna()

    # Encode text columns to number values
    for category in categorical:
        le = LabelEncoder()
        data[category] = le.fit_transform(data[category])
        label_encoders[category] = le

    for col in data.columns:
        if(col not in categorical):
            data[col] = (data[col].astype('float') - np.mean(data[col].astype('float')))/np.std(data[col].astype('float'))

    # print(data.describe())
    # print(data.describe(include='O'))

    processed_data = data

    # Get Training parameters
    if dataset == "German":
        target_col = data.columns[-1]
        x = data.drop(columns=target_col, axis=1)
        y = data[target_col].astype('int')
    elif dataset == "Australian":
        x = data.drop(14, axis=1)
        y = data[14].astype('int')
    elif dataset == "Japanese":
        x = data.drop(15, axis=1)
        y = data[15].astype('int')
    elif dataset == "Taiwan":
        x = data.drop('default_payment_next_month', axis=1)
        y = data['default_payment_next_month'].astype('int')
    elif dataset == "Polish":
        x = data.drop('class', axis=1)
        y = data['class'].astype('int')


    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size)
    x_train = pd.DataFrame(x_train)
    y_train = pd.DataFrame(y_train)
    y_test = pd.DataFrame(y_test)

    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    
    y_train = y_train[y_train.columns[0]].to_numpy()
    y_test = y_test[y_test.columns[0]].to_numpy()

    return (x_train, x_test, y_train, y_test)

X_train, X_test, y_train, y_test = preprocessing("German", german, test_size)

In [117]:
# Trnasform the input to tensor
X_test_tensor = base_torch.FloatTensor(X_test)
y_test_tensor = base_torch.tensor(y_test,dtype=base_torch.long)

In [118]:
# Calcualte Accuracy
from sklearn.metrics import accuracy_score

def accuracy(model,X,y):
    print(accuracy_score(model.predict(X),y))

In [119]:
accuracy(combined_model,X_test_tensor,y_test_tensor)

0.69
